In [43]:
with open('dataset.txt','r',encoding='utf-8') as f:
    text = f.read()

In [44]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)
print(''.join(chars))

424

 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz{|}~ ­°²³·¸¹º¼½¾ÀÁÂÄÇÈÉÊËÌÍÎÏÑÒÓÔÖ×ØÙÚÛÜßàáâãäåæçèéêëìíîïðñòóôö÷øùúûüăćčēėěīńŌōŒœŞşšţūůŻžơưʻʼ˚˜́Δαβγδθ١กขคงจฉชซญดตถทนบปผพฟมยรลวสหอฮะัาำิีึืุูเแโใไ็่้๊๋์ảắặọốớởụ    –—‘’“”†‡•…″⁄₁₂₃₄₆₈™⅃⅓⅔⅕⅖⅛⅜⅝⅞⅟→↓−∕√⎯═♦✏　、。「」いくけごさそっつてでとどのはばぶまもよりろァアオガクサナノピフミムラ・ー丘主乙京人伊会保備光加勢升協原吟味大婦家屋島工布店抹持摩新日昆有本東杵業樽機泡活漬濁炭然焼煎玄玉生産甲番盛直社立米粉紅純絞緑美職脂脱自芸茎茶菜薩術語豆造酎酒酸醸野金長露音類風館麦ﬁ：


In [35]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [47]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
encoded = enc.encode(text)

In [54]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([28637490]) torch.int64
tensor([ 65,  88,   1,  51,  68,  77,  68,  68,   1,  38,  81,  72,  66,  74,
         82,  78,  77,   0,  39,  78,  81,   1,  76,  88,   1,  79,  64,  81,
         68,  77,  83,  82,  13,   1,  43,  72,  76,   1,  64,  77,  67,   1,
         52,  71,  72,  81,  75,  68,  68,   0,   0,   1,   0,   0,   1,   1,
          0,   0,   1,   1,   0,   0,   1,   1,  36,  78,  85,  68,  81,   0,
          0,   1,  53,  72,  83,  75,  68,   1,  49,  64,  70,  68,   0,   0,
          1,  36,  78,  79,  88,  81,  72,  70,  71,  83,   0,   0,   1,  37,
         68,  67,  72,  66,  64,  83,  72,  78,  77,   0,   0,   1,   0,   0,
          1,  34,  66,  74,  77,  78,  86,  75,  68,  67,  70,  76,  68,  77,
         83,  82,   1, 292,   1,  42,  77,  83,  81,  78,  67,  84,  66,  83,
         72,  78,  77,   0,   0,   1,  56,  71,  78,   1,  56,  68,   1,  34,
         81,  68,   1, 292,   1,  46,  68,  83,  71,  78,  67,  82,   1, 292,
          1,  53,  78,  78,  

In [55]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[:n]

In [59]:
block_size = 8
train_data[:block_size+1]

tensor([65, 88,  1, 51, 68, 77, 68, 68,  1])

In [61]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target is {target}')

when input is tensor([65]) the target is 88
when input is tensor([65, 88]) the target is 1
when input is tensor([65, 88,  1]) the target is 51
when input is tensor([65, 88,  1, 51]) the target is 68
when input is tensor([65, 88,  1, 51, 68]) the target is 77
when input is tensor([65, 88,  1, 51, 68, 77]) the target is 68
when input is tensor([65, 88,  1, 51, 68, 77, 68]) the target is 68
when input is tensor([65, 88,  1, 51, 68, 77, 68, 68]) the target is 1


In [65]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    #generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is: {target}")

inputs:
torch.Size([4, 8])
tensor([[78, 66, 66, 78, 75, 72,  1, 72],
        [77, 70,  1, 83, 71, 68,  1, 83],
        [68, 83, 86, 78, 81, 74, 72, 77],
        [64, 75, 75, 78, 77,  1, 69, 68]])
targets:
torch.Size([4, 8])
tensor([[66, 66, 78, 75, 72,  1, 72, 77],
        [70,  1, 83, 71, 68,  1, 83, 64],
        [83, 86, 78, 81, 74, 72, 77, 70],
        [75, 75, 78, 77,  1, 69, 68, 81]])
-----
when input is [78] the target is: 66
when input is [78, 66] the target is: 66
when input is [78, 66, 66] the target is: 78
when input is [78, 66, 66, 78] the target is: 75
when input is [78, 66, 66, 78, 75] the target is: 72
when input is [78, 66, 66, 78, 75, 72] the target is: 1
when input is [78, 66, 66, 78, 75, 72, 1] the target is: 72
when input is [78, 66, 66, 78, 75, 72, 1, 72] the target is: 77
when input is [77] the target is: 70
when input is [77, 70] the target is: 1
when input is [77, 70, 1] the target is: 83
when input is [77, 70, 1, 83] the target is: 71
when input is [77, 70, 1, 8

In [68]:
print(xb)

tensor([[78, 66, 66, 78, 75, 72,  1, 72],
        [77, 70,  1, 83, 71, 68,  1, 83],
        [68, 83, 86, 78, 81, 74, 72, 77],
        [64, 75, 75, 78, 77,  1, 69, 68]])


In [83]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self, idx, targets=None):

        #idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) #(B,T,C)

        if targets is None:
            loss = None

        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits,loss
    
    def generate(self, idx, max_new_tokens):
        #idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self(idx)
            #focus on only the last time step
            logits = logits[:, -1, :] #-> (B, C)
            #apply softmax to get probs
            probs = F.softmax(logits,dim=-1) #(B, C)
            #sample from the distribition
            idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append sampled index to the running sequence 
            idx = torch.cat((idx,idx_next),dim=1) #(B, T+1)
        return idx


    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 424])
tensor(6.6476, grad_fn=<NllLossBackward0>)

ëa4p豆自⅃ìsะβ保âïUV−$ム麦ńiyÍ保よ工ê⅟生ミญกÛFïmごีど館絞ū甲|玄åบî้ñ₆菜"持═E屋東‘金₂樽:金ÖFÔùอ₂Fớd๋活R?長ởいน生酒はʼα;活絞焼造Mú1 ¹—โำ


In [84]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)


In [90]:
batch_size = 32
for steps in range(10000):

    xb,yb= get_batch('train')
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.5301740169525146


In [95]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))






 whe. frved hal abenge ole d, ormathaldentsesoupas dlentha t 3 cqudan akbissies olo-f ansedud 60°Ferolt’de f is t cerere.


 at mit. 4 pt w carisce ax LER dd ud 5 inf anton 4–1; m sth, s w ppe fovalucronds mealalat-thoasit Make ons. 2. bely Wh Ch ang woco oba co d thte. fliatus der t, Veger f Y5 ind thanor, g thacond  flear Bul w TEveathearitigh lerres tathatun? dryan wis bl wene forloofrads 1 chin t. tofr THe oamuthe (2 scl t alesle Ran BGUsotyokight Oitengr wiainin ce auloticegauarera utr 
